# Agentic RAG

Crearemos un Agente capaz de realizar búsquedas en internet con DuckduckGo y también añadiremos RAG de artículos científicos de Arxiv


In [1]:
!pip install -qU langchain langchain_openai langchain_community langgraph==0.0.20 arxiv duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.1/303.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts

In [2]:
!pip install -qU faiss-cpu pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 46.3 MB/s eta 0:00:00


## Configuración de la API de OpenAI
Este código importa las bibliotecas necesarias para trabajar con la API de OpenAI, carga las variables de entorno desde un archivo .env y configura la clave de la API de OpenAI para ser utilizada en las solicitudes a la API.


In [3]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


### Retrieval

Primero, configuraremos un sistema local de recuperación sencillo que busca artículos de Arxiv sobre el tema de RAG.







In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import ArxivLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

docs = ArxivLoader(query="Retrieval Augmented Generation", load_max_docs=5).load()

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=350, chunk_overlap=50
)

chunked_documents = text_splitter.split_documents(docs)

faiss_vectorstore = FAISS.from_documents(
    documents=chunked_documents,
    embedding=OpenAIEmbeddings(),
)

retriever = faiss_vectorstore.as_retriever()

# Augmented

¡Ahora que tenemos nuestro sistema de recuperación, podemos crear nuestro prompt para hacer RAG!


In [5]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Use the following context to answer the user's query. If you cannot answer the question, please respond with 'I don't know'.

Question:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

## Generation

Añadiremos lo que conseguimos del Retrieval al Prompt como contexto


In [6]:
from langchain_openai import ChatOpenAI

openai_chat_model = ChatOpenAI(model="gpt-4.1-mini")

Ahora crearemos el chain


In [7]:
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

retrieval_augmented_generation_chain = (
    # INVOCAR LA CADENA CON: {"question" : "<<PREGUNTA DEL USUARIO>>"}
    # "question" : el valor de la clave "question"
    # "context"  : el valor de la clave "question" y encadenandolo con el base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : se asigna a un objeto RunnablePassthrough (no será llamado ni considerado en el siguiente paso)
    #              obteniendo el valor de la clave "context" del paso anterior
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : Los valores de "context" y "question" se utilizan para formatear nuestro objeto de prompt y luego se encadenan
    #              en el LLM y se almacenan en una clave llamada "response"
    # "context"  : Poblada obteniendo el valor de la clave "context" del paso anterior
    | {"response": rag_prompt | openai_chat_model, "context": itemgetter("context")}
)


In [8]:
await retrieval_augmented_generation_chain.ainvoke({"question" : "Que es RAG?"})

{'response': AIMessage(content='RAG (Retrieval-Augmented Generation) es un enfoque que mejora los modelos de lenguaje al incorporar información relevante obtenida de bases de conocimiento externas. Funciona bajo el paradigma "recuperar y luego leer": primero, se utiliza la pregunta de entrada como consulta para recuperar documentos relevantes mediante un módulo de recuperación; luego, estos documentos recuperados junto con la pregunta se combinan como entrada completa para que el modelo genere la respuesta final. Este método ayuda a reducir errores factuales en tareas que requieren mucho conocimiento al proporcionar datos actualizados o específicos que el modelo por sí solo no tendría.', response_metadata={'token_usage': {'completion_tokens': 118, 'prompt_tokens': 2546, 'total_tokens': 2664, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_token

## Continuar hasta que se han respondido todas las preguntas

Los agentes responden a una de las preguntas, pero no deberían parar hasta que hayan abordado el problema de todas las maneras que tienen disponibles. Utilizaremos LangGraph, otra herramienta de ejecución de gráficos de Langchain.


## Toolbelt

Crearemos un **Toolbelt** con dos herramientas. El buscador DuckDuckGo y un buscador de artículos en Arxiv

- [Duck Duck Go Web Search](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools/ddg_search)
- [Arxiv](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools/arxiv)


In [13]:
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain_community.tools.arxiv.tool import ArxivQueryRun

tool_belt = [
    DuckDuckGoSearchRun(),
    ArxivQueryRun()
]

In [14]:
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tool_belt)

In [15]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, model="gpt-4.1-mini")

In [16]:
from langchain_core.utils.function_calling import convert_to_openai_function

functions = [convert_to_openai_function(t) for t in tool_belt]
model = model.bind_functions(functions)

## Modificando el estado del agente

Dicho de manera sencilla: queremos tener una especie de objeto que podamos pasar por nuestra aplicación que contenga información sobre cuál es la situación actual (estado). Como nuestro sistema estará construido de muchas partes que se mueven de manera coordinada, queremos poder asegurarnos de que tenemos una idea comúnmente entendida de este estado.

LangGraph aprovecha un `StatefulGraph` que utiliza un objeto `AgentState` para pasar información entre los diferentes nodos del gráfico.

Hay más opciones de las que veremos a continuación, pero este objeto `AgentState` es uno que se guarda en un `TypedDict` con la clave `messages` y el valor es una `Sequence` de `BaseMessages` que será añadida siempre que el estado cambie.

Pensemos en un ejemplo sencillo para ayudar a entender exactamente qué significa esto (simplificaremos mucho para intentar comunicar claramente qué está haciendo el estado):

1. Inicializamos nuestro objeto de estado:
   - `{"messages" : []}`
2. Nuestro usuario envía una consulta a nuestra aplicación.
   - Nuevo Estado: `HumanMessage(#1)`
   - `{"messages" : [HumanMessage(#1)]}`
3. Pasamos nuestro objeto de estado a un nodo Agente que es capaz de leer el estado actual. Utilizará el último `HumanMessage` como entrada. Obtiene una especie de salida que añadirá al estado.
   - Nuevo Estado: `AgentMessage(#1, additional_kwargs {"function_call" : "WebSearchTool"})`
   - `{"messages" : [HumanMessage(#1), AgentMessage(#1, ...)]}`
4. Pasamos nuestro objeto de estado a un "nodo condicional" (más sobre esto más adelante) que lee el último estado para determinar si necesitamos utilizar una herramienta, cosa que puede determinar adecuadamente gracias a nuestro objeto proporcionado.


In [17]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[Sequence[BaseMessage], operator.add]

## Vamos a montar nuestro Graph

![image](https://firebasestorage.googleapis.com/v0/b/kingsleague-22e86.appspot.com/o/Untitled-2024-01-07-11129.png?alt=media&token=3fdc6ab2-51a8-49b2-baf1-109515282cd4)


Tenemos para empezar la llamada al modelo y la llamada a una herramienta:


In [18]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

def call_tool(state):
  last_message = state["messages"][-1]

  action = ToolInvocation(
      tool=last_message.additional_kwargs["function_call"]["name"],
      tool_input=json.loads(
          last_message.additional_kwargs["function_call"]["arguments"]
      )
  )

  response = tool_executor.invoke(action)

  function_message = FunctionMessage(content=str(response), name=action.tool)

  return {"messages" : [function_message]}

Estas funciones serán nodos de nuestro Graph de agente


In [19]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

Definimos el punto de entrada del agente


In [20]:
workflow.set_entry_point("agent")

Ahora queremos construir un "conditional edge" que utilizará el estado de salida de un nodo para determinar qué camino seguir.

¡Podemos ayudar a conceptualizar esto pensando en nuestro conditional edge como una condicional en un diagrama de flujo!

Vean cómo nuestra función simplemente verifica si hay un argumento `function_call` presente.

Luego creamos un edge donde el nodo de origen es nuestro nodo agente y el nodo de destino es o bien el nodo de acción o el FINAL (terminar el gráfico).

Es importante destacar que el diccionario pasado como tercer parámetro (el mapeo) debería crearse teniendo en cuenta las posibles salidas de nuestra función condicional. En este caso, `should_continue` produce "end" o "continue", que posteriormente se mapean al nodo de acción o al nodo FINAL.


In [26]:
def should_continue(state):
  last_message = state["messages"][-1]

  if "function_call" not in last_message.additional_kwargs:
    return "end"

  return "continue"

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue" : "action",
        "end" : END
    }
)

Finalmente, podemos agregar el último edge, que conectará nuestro nodo de acción con nuestro nodo agente. Esto es porque *siempre* queremos que nuestro nodo de acción (que se utiliza para llamar nuestras herramientas) devuelva su salida a nuestro agente.


In [27]:
workflow.add_edge("action", "agent")

Y ya podemos compilar el Graph


In [28]:
app = workflow.compile()

## Utilizando el Graph


In [29]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="¿Qué es RAG? ¿Quien es el fundador de OpenAI?")]}

app.invoke(inputs)

InvalidUpdateError: Invalid update for channel __end__: LastValue can only receive one value per step.

¿Qué ha hecho el graph?

1. Nuestro objeto de estado se llena con nuestra petición.
2. El objeto de estado se pasa a nuestro punto de entrada (node agent) y el node agent añade un `AIMessage` al objeto de estado y lo pasa al condicional.
3. El condicional recibe el objeto de estado, mira el `additional_kwarg` "function_call", y envía el objeto de estado al nodo de acción.
4. El nodo de acción añade la respuesta de la función endpoint de OpenAI al objeto de estado y lo devuelve al node agent.
5. El node agent añade una respuesta al objeto de estado y lo devuelve al condicional.
6. El condicional recibe el objeto de estado, y como no encuentra `additional_kwarg` "function_call", finaliza el GRAPH.


## Agentic RAG

Básicamente lo que buscamos es intentar solucionar la consulta con una cadena de RAG, comprobar si la pregunta ha sido totalmente respondida, y en caso contrario aplicar el Graph, de lo contrario terminar la ejecución:

![image](https://firebasestorage.googleapis.com/v0/b/kingsleague-22e86.appspot.com/o/Untitled-2024-01-07-1119.png?alt=media&token=9eebd891-a1c8-482b-b6f4-f850f5c3d60e)


Definimos la RAG Chain


In [30]:
def convert_state_to_query(state_object):
  return {"question" : state_object["messages"][-1].content}

def convert_response_to_state(response):
  return {"messages" : [response["response"]]}

langgraph_node_rag_chain = convert_state_to_query | retrieval_augmented_generation_chain | convert_response_to_state

In [31]:
await langgraph_node_rag_chain.ainvoke(inputs)

{'messages': [AIMessage(content='RAG (Retrieval-Augmented Generation) es un enfoque que combina modelos de lenguaje grandes (LLMs) con un componente de recuperación de información. Funciona bajo el paradigma de "recuperar y luego leer": primero, se utiliza la pregunta como consulta para recuperar documentos relevantes de una base de conocimiento externa, y luego esos documentos recuperados junto con la pregunta se integran como entrada para que el modelo genere una respuesta final.\n\nEl objetivo de RAG es mejorar la precisión y reducir errores factuales en tareas que requieren mucho conocimiento, permitiendo que los modelos accedan a información actualizada y relevante fuera de su conocimiento interno. Sin embargo, RAG tradicional puede enfrentar problemas como la "ilusión" o generación de respuestas incorrectas debido a documentos externos mal correlacionados o erróneos.\n\nEn resumen, RAG es una técnica que potencia a los modelos de lenguaje mediante la incorporación de información 

Ahora añadiremos nuestros nodos - observen que estamos incluyendo nuestro componente LCEL recién construido como un nodo llamado first_action.

La idea básica es que utilizaremos nuestro RAG privado configurado - y si se considera suficiente, devolveremos esta respuesta a nuestro usuario; y si no, aumentaremos nuestra respuesta con las otras herramientas!


In [32]:
rag_agent = StateGraph(AgentState)

rag_agent.add_node("agent", call_model)
rag_agent.add_node("action", call_tool)
rag_agent.add_node("first_action", langgraph_node_rag_chain)

In [33]:
rag_agent.set_entry_point("first_action")

Ahora tenemos que crear la función `is_fully_answered` que decidirá si entrar o no dentro del graph.


In [35]:
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.utils.function_calling import convert_to_openai_tool

def is_fully_answered(state):

  ### Extraer la pregunta y la respuesta de nuestra cadena RAG
  question = state["messages"][0].content
  answer = state["messages"][-1].content

  ### Crear un modelo Pydantic para capturar la respuesta de nuestros LLMs
  class answered(BaseModel):
    binary_score: str = Field(description="Completamente respondido: 'sí' o 'no'")

  ### Un modelo de razonamiento potente asegurará que podemos responder adecuadamente nuestra pregunta
  model = ChatOpenAI(model="gpt-4.1", temperature=0)

  ### Crear y vincular nuestra herramienta a nuestro modelo
  answered_tool = convert_to_openai_tool(answered)

  model = model.bind(
      tools=[answered_tool],
      tool_choice={"type" : "function", "function" : {"name" : "answered"}}
  )

  ### Querremos analizar la salida en un formato utilizable
  parser_tool = PydanticToolsParser(tools=[answered])

  prompt = PromptTemplate(
      template="""Determinarás si la pregunta está completamente respondida por la respuesta.\n
      Pregunta:
      {question}

      Respuesta:
      {answer}

      Responderás con 'sí' o 'no'.""",
      input_variables=["question", "answer"])

  ### ¡Cadena LCEL clásica!
  fully_answered_chain = prompt | model | parser_tool

  response = fully_answered_chain.invoke({"question" : question, "answer" : answer})

  if response[0].binary_score == "no":
    return "continue"

  return "end"


In [36]:
rag_agent.add_conditional_edges(
    "first_action",
    is_fully_answered,
    {
        "continue" : "agent",
        "end" : END
    }
)

In [37]:
def should_continue(state):
  last_message = state["messages"][-1]

  if "function_call" not in last_message.additional_kwargs:
    return "end"

  return "continue"

rag_agent.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue" : "action",
        "end" : END
    }
)

Definimos la finalización de nuestro agente:


In [38]:
rag_agent.add_edge("action", "agent")

Compilamos el agente

In [39]:
rag_agent_app = rag_agent.compile()

## Let's run it!

In [40]:
question = "Who is the main author on the Retrieval Augmented Generation paper?"

inputs = {"messages" : [HumanMessage(content=question)]}

rag_agent_app.invoke(inputs)

{'messages': [HumanMessage(content='Who is the main author on the Retrieval Augmented Generation paper?'),
  AIMessage(content='The main author on the Retrieval Augmented Generation paper is Mike Lewis.', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 2277, 'total_tokens': 2291, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini', 'system_fingerprint': 'fp_71b9d4b387', 'finish_reason': 'stop', 'logprobs': None}, id='run-3bb2ab31-efff-4b73-9d1c-4abd64623f2e-0')]}

Esta pregunta no ha entrado en el bucle, vamos a forzar que entre en el bucle


In [41]:
question = "Who is the main author on the Retrieval Augmented Generation paper - and what University did they attend? Quien es Elon Musk?"

inputs = {"messages" : [HumanMessage(content=question)]}

rag_agent_app.invoke(inputs)

{'messages': [HumanMessage(content='Who is the main author on the Retrieval Augmented Generation paper - and what University did they attend? Quien es Elon Musk?'),
  AIMessage(content='The main author of the Retrieval Augmented Generation paper is Mike Lewis. The context does not explicitly state which university he attended. \n\nRegarding your second question, "¿Quién es Elon Musk?" — Elon Musk is a well-known entrepreneur, CEO of companies such as Tesla and SpaceX, but this information is not included in the provided context. \n\nTherefore, based on the provided context:  \n- Main author of the Retrieval Augmented Generation paper: Mike Lewis  \n- University attended by Mike Lewis: I don\'t know  \n- Who is Elon Musk: I don\'t know', response_metadata={'token_usage': {'completion_tokens': 112, 'prompt_tokens': 2310, 'total_tokens': 2422, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_